# fMRI Problem Set 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline

## Problem 1: Convolution

In fMRI analysis, we usually assume that the BOLD signal is the result of a linear time-invariant (LTI) system [(Boynton et al., 1996)](http://www.jneurosci.org/content/16/13/4207); in other words, we assume (1) the shape of the hemodynamic response is constant across time, and (2) the responses to successive stimuli superpose linearly (additively). 

Assumption #2 justifies using convolution in generating the predicted BOLD signal from a set of stimulus events, $x$, and a hemodynamic response function, $h$. Conceptually, what convolution does is add the entire HRF shape starting wherever there is a non-zero entry. 

In this first exercise, demonstrate that `np.convolve` is equivalent to a `for loop` iterating over the non-zero entries of an events array and adding the HRF beginning at those entries.

In [ ]:
from fmritools.hrf import spm_hrf
np.random.seed(0)

## Define events array.
x = np.zeros(100)
ix = np.random.choice(np.arange(100), 20, replace=False) # Randomly choose 20 events.
x[ix] = 1

## Define HRF.
hrf = spm_hrf(1)

## FOR LOOP CODE GOES HERE.


## CONVOLUTION CODE GOES HERE.


## Demonstrate equivalence.
# print(np.allclose(bold_sliding, bold_convolve))

## Problem 2: Design Matrix

Write a function that automates the generation of an fMRI experiment design matrix, completing all of the steps discussed in class (e.g. defining times, convolution, downsampling). You can structure the function however you like, but at the minimum it must accept as arguments:

1. Repetition time (TR)
2. Total number of observations
3. Stimulus events: onsets (in seconds), offsets (in seconds), conditions

The function must return a matrix of predicted BOLD regressors. (You may also think of adding additional arguments, allowing the user to change the expected HRF shape, normalization of regressors, etc.). For your convenience, the file *fmri-ps1-02.npz* contains two experiment designs and their respective design matrices for testing purposes. Both used the `spm_hrf` function.

You will not be graded on the efficiency of the code, but its documentation. The function should include a docstring complete with a `parameters` and `returns` section (see [here](https://numpydoc.readthedocs.io/en/latest/format.html) for examples). Internally, the code itself should have enough documentation to let me know what each section of code is doing.

### Write design matrix function

### Test design matrix function

In [ ]:
## Load data.
npz = np.load('fmri-ps1-02.npz')

## Experiment 1.
experiment_1 = npz['experiment_1'].item()

## Experiment 2.
experiment_2 = npz['experiment_2'].item()

## Problem 3: Mis-modeling the HRF, Part 1

In a brief write-up, [Grinband et al. (2008)](https://www.ncbi.nlm.nih.gov/pubmed/18775784) pointed out a potential methodological issue in how fMRI experiments involving reaction times (RTs) were being modeled. In electrophysiological studies, there is considerable evidence for neural activity up to the point of response. Many fMRI studies, however, were modeling reaction time trials uniformly, mapping the length of the neural boxcars to the trial length and not the RT length itself. This is problematic as modeling fast and slow RTs with identical HRF responses is potentially confounding changes in BOLD amplitude with changes in BOLD duration (i.e. time-on-task effect). Thus, [Grinband et al. (2008)](https://www.ncbi.nlm.nih.gov/pubmed/18775784) recommended a **variable epoch** approach, where RT trials were modeled with boxcars the length of the RT response.

In this problem, we will demonstrate that the variable epoch approach can better recover true changes in the BOLD amplitude. We have simulated for you data from a simple rapid event related design experiment. The experiment is 320 total acquisitions with TR=2. There are two conditions with 80 trials each, and trials last 2s. 

Importantly, the true BOLD signal change in both conditions is the assume (~1.25%). However, the response times across conditions differ: the average response time in the first condition is approximately 800ms, whereas the average response time in the second condition is approximately 1200ms.

### [3a] Show the difference in reaction times.
Load the experiment data, stored in *fmri-ps1-03.npz*. Confirm there is a difference in reaction times. (You can plot the difference or use `scipy.stats.ttest_ind`.)

### [3b] Mis-model the BOLD response

Using the function you defined in Problem 2, generate the expected BOLD response assuming the boxcar offsets is at the end of the trial (i.e. 2s from trial onset). When performing your regression, don't forget to include an intercept term (column of ones).

Are the results biased (i.e. differ than ~1.25)? If so, which condition is worse affected?

### [3c] Using function defined above, model with variable boxcars.

Again using the function you defined in Problem 2, generate the expected BOLD response assuming the boxcar offsets is at the time of response. When performing your regression, don't forget to include an intercept term (column of ones).

Are the results still biased (i.e. differ than ~1.25)?

## Problem 4: FIR Design Matrix

Write a function that automates the generation of an FIR design matrix, completing all of the steps discussed in class (e.g. defining times, convolution, downsampling). You can structure the function however you like, but at the minimum it must accept as arguments:

1. Repetition time (TR)
2. Total number of observations
3. Length of FIR window (in TRs)
4. Stimulus events: onsets (in seconds), conditions

The function must return a binary FIR matrix. For your convenience, the file *fmri-ps1-04.npz* contains two experiment designs and their respective design matrices for testing purposes. 

You will not be graded on the efficiency of the code, but its documentation. The function should include a docstring complete with a `parameters` and `returns` section (see [here](https://numpydoc.readthedocs.io/en/latest/format.html) for examples). Internally, the code itself should have enough documentation to let me know what each section of code is doing.

### Write FIR matrix function

### Test FIR matrix function

In [ ]:
## Load data.
npz = np.load('fmri-ps1-04.npz')

## Experiment 1.
experiment_1 = npz['experiment_1'].item()

## Experiment 2.
experiment_2 = npz['experiment_2'].item()

## Problem 5:  Mis-modeling the HRF, Part 2

After [Grinband et al. (2008)](https://www.ncbi.nlm.nih.gov/pubmed/18775784), [Yarkoni et al. (2009)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0004257) dug even deeper into the modeling of fMRI reaction time experiments. They pointed out that prolonged reaction times may not just extend the neural boxcar, but fundamentally change the shape of the HRF itself. Using a series of FIR design analyses, [Yarkoni et al. (2009)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0004257) found that across a variety of experiments, prolonged reaction times did indeed change the shape of the HRF.

In this final problem, we will recreate the results of this paper. We have simulated for you data from the same rapid event related design experiment as above (i.e. 2 conditions, 80 trials in each). The only change is that the experiment was recollected at TR=1.

Use the FIR design matrix function you wrote in Problem 4 to generate the corresponding FIR design matrix for this experiment. Use a kernel window of 16 (i.e. modeling 16s of the HRF). When performing your regression, don't forget to include an intercept term (column of ones).

Are there differences in the HRF between conditions?